## RUNNNIG THE LDA

So we continue from where we left in the previous notebook. We load up the articles.

In [2]:
#taking the input of number of topics required by the user
k = input("Enter the number of topics for topic model\n")

Enter the number of topics for topic model
5


## approach 2
tried a different approach which didn't work out well so this one.

In [32]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

import json
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/himanshu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [54]:
#reading in the data. data is in list format
#each element of list is a document
with open("articles.json") as file:
    data= json.load(file)

In [55]:
#lemmatizing and stemming the data
#this will help us to keep only important words like NOUN,VERB,
#and would also remove too short or long words
#would convert inflicted forms to base
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

stemmer = SnowballStemmer("english")

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [34]:
#printing original form of data
data[0]

'NoneKRAUS-George E., 88, of Scottsdale, AZ, formerly of New York, died February 13. Beloved husband of Bobbi, cherished father and father-in-law to Marlene and Steve Morris of New York and Leslie Kraus and Susan Rosenthal Kraus of Minnesota. Loving grandfather of David and Gary Morris and Erica Rosenthal Kraus. His goodness, wisdom and sense of humor will be forever appreciated. Contributions may be made to the charity of your choice.Paid Notice: Deaths KRAUS, GEORGE E.'

In [35]:
#printing sample of processed data
print(preprocess(data[0]))

['nonekraus', 'georg', 'scottsdal', 'york', 'die', 'februari', 'belov', 'husband', 'bobbi', 'cherish', 'father', 'father', 'marlen', 'steve', 'morri', 'york', 'lesli', 'kraus', 'susan', 'rosenth', 'kraus', 'minnesota', 'love', 'grandfath', 'david', 'gari', 'morri', 'erica', 'rosenth', 'kraus', 'good', 'wisdom', 'sens', 'humor', 'forev', 'appreci', 'contribut', 'chariti', 'choic', 'pay', 'notic', 'death', 'kraus', 'georg']


In [36]:
#processing each doc and adding it to list of processed_docs
processed_docs = [preprocess(doc) for doc in data]

In [56]:
#creating a dictionary maping each word to id
#note that this is very small data
#else we should have used MapReduce
dictionary = gensim.corpora.Dictionary(processed_docs)

In [58]:
#printing a few examples
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 4:
        break

0 appreci
1 belov
2 bobbi
3 chariti
4 cherish


In [93]:
#right now dont feel I have use of keep_n parameter as number of docs is very low
#not keeping words that appear in less than 5 documens
#not keeping words appearing in more than 50 percent documents
#maybe I would have set no_above lower if we had a larger dataset
dictionary.filter_extremes(no_below=5, no_above=0.5)

In [94]:
#document representation in bag of words format
#basically each doc is now just words and their frequency without any importance to position
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [95]:
#printing some values
bow_corpus[0][:5]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

In [96]:
#reqiire these for tfidf model
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.19186139221299328),
 (1, 0.2220538093360031),
 (2, 0.262219913081204),
 (3, 0.2148540519150605),
 (4, 0.16748819074891697),
 (5, 0.1420975834984306),
 (6, 0.10089215183985274),
 (7, 0.1709932920157968),
 (8, 0.32195043586624406),
 (9, 0.16748819074891697),
 (10, 0.262219913081204),
 (11, 0.2392272533791368),
 (12, 0.38372278442598656),
 (13, 0.127322087003716),
 (14, 0.24976106193256148),
 (15, 0.14955966175210916),
 (16, 0.1420975834984306),
 (17, 0.11360935676697848),
 (18, 0.10625446989686663),
 (19, 0.18714679931850206),
 (20, 0.2392272533791368),
 (21, 0.262219913081204)]


In [103]:
#training LDA model based on BOW
lda_model_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary,batch=False, passes=10, workers=None, iterations=1000)

In [104]:
#getting visualization tool ready
vis_lda_bow = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)

## USING THE GRAPH

We can see that our topics are very neatly segregated into relevant clusters.
Please take the cursor over a topic number to see the relevant keywords on the right

In [105]:
vis_lda_bow

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.003708  0.038644       1        1  23.933836
3     -0.006289  0.127645       2        1  22.303640
0      0.053381 -0.043878       3        1  20.899012
1     -0.159145 -0.060803       4        1  17.671202
2      0.108345 -0.061608       5        1  15.192309, topic_info=     Category        Freq      Term       Total  loglift  logprob
term                                                             
491   Default   91.000000   billion   91.000000  30.0000  30.0000
246   Default  118.000000   clinton  118.000000  29.0000  29.0000
1032  Default   72.000000     pilot   72.000000  28.0000  28.0000
44    Default  111.000000   percent  111.000000  27.0000  27.0000
275   Default  125.000000   million  125.000000  26.0000  26.0000
34    Default  183.000000  american  183.000000  25.0000  25.0000
150   Default  118.000000    street  118.000000  24.0000  24.0000
180   Default  125.000000      citi  125.000000  23.0000  23.0000
1001  Default   43.000000     stock   43.000000  22.0000  22.0000
1026  Default   61.000000    market   61.000000  21.0000  21.0000
17    Default   42.000000     notic   42.000000  20.0000  20.0000
410   Default   36.000000      film   36.000000  19.0000  19.0000
1363  Default   42.000000      drug   42.000000  18.0000  18.0000
37    Default  160.000000   compani  160.000000  17.0000  17.0000
739   Default   56.000000      rise   56.000000  16.0000  16.0000
1143  Default   59.000000    school   59.000000  15.0000  15.0000
51    Default   29.000000   theater   29.000000  14.0000  14.0000
624   Default   69.000000  democrat   69.000000  13.0000  13.0000
6     Default   59.000000     death   59.000000  12.0000  12.0000
1009  Default   41.000000    airlin   41.000000  11.0000  11.0000
194   Default   44.000000      game   44.000000  10.0000  10.0000
918   Default   37.000000      sale   37.000000   9.0000   9.0000
23    Default   51.000000     avenu   51.000000   8.0000   8.0000
53    Default  126.000000      work  126.000000   7.0000   7.0000
471   Default   36.000000    review   36.000000   6.0000   6.0000
903   Default   30.000000    profit   30.000000   5.0000   5.0000
1250  Default   37.000000    immigr   37.000000   4.0000   4.0000
855   Default   52.000000    chines   52.000000   3.0000   3.0000
127   Default   63.000000      play   63.000000   2.0000   2.0000
476   Default   44.000000     share   44.000000   1.0000   1.0000
...       ...         ...       ...         ...      ...      ...
1139   Topic5    6.608970    mental    9.418703   1.5301  -6.4111
113    Topic5    5.241512     marri    7.547408   1.5198  -6.6429
96     Topic5    5.860111   instanc    8.457848   1.5175  -6.5313
43     Topic5   16.425480     music   24.448135   1.4867  -5.5007
537    Topic5   18.298782   exhibit   28.255898   1.4499  -5.3927
123    Topic5   27.354931   perform   46.258007   1.3590  -4.9906
682    Topic5   12.220706      bodi   18.831535   1.4520  -5.7964
867    Topic5   17.432753    direct   30.096575   1.3383  -5.4411
386    Topic5   17.482101      name   30.231869   1.3367  -5.4383
231    Topic5   19.195255      star   34.109646   1.3095  -5.3448
91     Topic5   11.971203    garden   18.819283   1.4320  -5.8170
53     Topic5   49.489285      work  126.128616   0.9488  -4.3977
979    Topic5   15.398360     figur   28.307175   1.2755  -5.5652
127    Topic5   26.041170      play   63.874084   0.9871  -5.0398
414    Topic5   12.880006      imag   23.485901   1.2837  -5.7438
387    Topic5   22.611837      open   63.606407   0.8501  -5.1810
181    Topic5   24.076424     close   71.836868   0.7912  -5.1183
106    Topic5   32.644642      like  131.215256   0.4932  -4.8138
109    Topic5   19.715563      live   52.162247   0.9114  -5.3181
514    Topic5   19.582655     photo   52.108658   0.9057  -5.3249
203    Topic5   15.290556      head   35.104713   1.0533  -5.5723
1175   Topic5  

In [108]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_bow, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3533569216786051
